In [23]:
import dgl
import torch
import torch.nn as nn
from dgl.nn import GINEConv
import numpy as np

In [15]:
g=dgl.graph(([0,1,2,1],
             [1,2,1,0]))
in_feat=3
out_feat=2
n_feat=torch.randn(g.num_nodes(),in_feat)
e_feat=torch.randn(g.num_edges(),in_feat)

In [17]:
print(n_feat)
print(e_feat)
conv=GINEConv(nn.Linear(in_feat,out_feat))
res=conv(g,n_feat,e_feat)
print(res)

tensor([[ 0.7645, -1.0923,  0.3626],
        [-0.2123,  1.4085, -1.1527],
        [ 0.6988, -0.6517,  1.3276]])
tensor([[-0.8593, -0.3838, -0.2183],
        [-1.6036,  1.3587,  0.1783],
        [-0.6317,  1.0664,  0.4102],
        [ 0.4911, -0.0301,  1.0445]])
tensor([[0.1455, 0.6236],
        [0.3387, 0.3294],
        [0.0392, 1.0487]], grad_fn=<AddmmBackward0>)


In [38]:
a=torch.randn(10,10)
b=torch.randn(10,10)
c=a.detach()

In [39]:
cos=nn.CosineSimilarity()
output=cos(a,b)
print(output)

tensor([ 0.1330,  0.3394, -0.2787, -0.3231, -0.2621,  0.3862, -0.1411,  0.5284,
         0.2717, -0.1268])


In [44]:
torch.backends.cudnn.enabled

True